# EDS 220 Fall 2022: Visualizing Landsat data using Google Earth Engine

## Part I: True and False Color Images

In this notebook, we'll be learning how to use Google Earth Engine to create map-based visualizations of remote sensing data. We'll be working with Landsat data; in case you're curious, the full Landsat GEE catalog is available here:

https://developers.google.com/earth-engine/datasets/catalog/landsat



### Configure your environment (if necessary)

**REMINDER:** If you are working on a local Python install, make sure you have followed the instructions for installing GEE on your machine:
https://docs.google.com/document/d/1xviSPmI8iFa1Y31TVO4DiKkbMnh-71iVoqQ_wymc7pc/edit

before attempting to run this notebook. 

__Everything should work correctly on the Bren Taylor server - if not, please let us know!__

**1. Import required packages**

First, let's import the packages that will be needed for this exercise: `ee`, the package containing the Google Earth Engine codes, `geemap`, which holds the mapping functionality of GEE, and our good friend `pandas`.

In [2]:
# Import packages
import ee
import geemap
import pandas as pd

**2. Initialize the GEE environment**

Next, we have to make sure that our authentication credentials for GEE are active, and that GEE has been initialized on the remote servers. This is accomplished using the `ee.Authenticate()` and `ee.Initialize()` commands. These are generally handy if your connection to the GEE API has been idle for a while or got disrupted somehow, since they allow you to re-authenticate (`ee.Authenticate()`) and re-establish your connection with the API server (`ee.Initialize()`):

In [ ]:
# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

### Read in Landsat 8 data

Next, we need to actually import data from Landsat. There are multiple Landsat missions which have launched over the years: let's use Landsat 8 here, since it is the most recent dataset available as of this writing. 

Landsat provides information in several processing 'tiers':

   - **Tier 1** data are considered high-quality
   - **Tier 2** data don't pass muster to get into Tier 1
   - **Real-Time (RT)** data haven't been processed into either tier yet
    
A full description of the Landsat data organizational structure in GEE can be found at: https://developers.google.com/earth-engine/guides/landsat

The below example reads in data from:

 - The Landsat 8 mission (LC08)
 - Data Collection 1 (C01); this is the first "official" collection of Landsat data ingested into GEE, which will eventually be superseded by Collection 2 but will be maintained through the end of 2022
 - Tier 1 data (T1), the highest data quality 
 
Note that the data we will be reading in here are _top-of-atmosphere (TOA) reflectances_, estimates of the radiation reflected at the top of Earth's atmosphere.

The first thing we'll do is to read in the Tier 1-Real Time dataset, using the `ee.ImageCollection` function:

The code chunk below also displays some metadata on the Landsat image collection using the `first()` method to extract the first image in the collection, then the `getInfo()` method to display information on the bands and other metadata included in that image. These steps are also described in more detail in the HW1 repo!

In [ ]:
# Load the Landsat 8 Tier 1 TOA image collection
gdat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# show some metadata
testimg=gdat.first()
testimg.getInfo()

The `gdat` object above is now an ImageCollection, which is pretty much exactly what it sounds like: a data structure made up of all of the Landsat 8 Collection 1 Tier 1-Real Time images. As you can imagine, this is a LOT of information!

Let's figure out what's going on with this data a little bit. It's helpful to look at the *metadata* for any new data source; the easiest way to do that in this case is to look at the Web documentation:

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description

Take a look at the URL above; read the data *description* then note the information on *bands* and *properties*. 

- **Bands** The Bands tab will show you the names and wavelengths of the different spectral bands that Landsat 8 is sensing in.
- **Properties** These are other things that are useful to know about any given piece of data: things like the data processing level, estimated cloud cover, and the PATH and ROW of a given scene. (Recall that these are standardized for the Landsat missions, onto a particular lat/lon grid!)

It's not important to understand the details of every band and property; what matters is knowing where to find this information when you do need it!

## Select a Region

Generally speaking, we're not interested in looking at the entire Landsat archive all at once: instead, we usually want to select a portion of the available data corresponding to a region (and often time period) of interest. Because Landsat (and satellite information in general) is created in repeating swaths slicing across the planet, finding only the data which correspond to a particular spatial and temporal region can be a complex task. Luckily, there are various algorithms which will filter Landsat data for us! In the GEE implementation, this is done using `ee.Filter()` and related functions.

Filtering spatially can take several forms. You can:
- **Select a specific point of interest**
- **Define lat/lon bounds for a region of interest**
or
- **Define your own irregular polygon** which could be a shapefile, or any kind of arbitrary shape defined by a set of lat and lon boundary points.

As a first step, let's select a point of interest: say, Santa Barbara. The `ee.Geometry.Point()` method allows us to define a point using lat/lon coordinates, which we can then use to select all the Landsat images which intersect that point.

In [5]:
pt = ee.Geometry.Point([-119.69, 34.42])   # Point corresponding to Santa Barbara, CA

The `ee.filterBounds()` method then allows us to filter the ImageCollection using the point we've just defined.

In [6]:
gdat_filt = gdat.filterBounds(pt)

### Plot all available data within a region

Now we have filtered the Landsat data to contain only those scenes which include Santa Barbara. Now what? Well, maybe we should look at them! GEE will allow you to plot all images within an ImageCollection; this is done by adding the data as a layer, the same way we did for the ERA data earlier.

The below example combines data from multiple *bands* in the ImageCollection, into a "true color" image. We do this by selecting the red (band B4), green (band B3), and blue (band B2) wavelength data, then passing them as *parameters*. By default, GEE will plot these as RGB colors and plot using a color range from 0 to 1; you can change this by passing `max` and `min` values to the parameter array below. Here I have used a max value of 0.3, which the GEE developer guides specify as a reasonable max value.

NOTE: the below code will not produce output, and that is ok! It's just creating a data structure with the appropriate visualization information, stored in `visParams`.

In [29]:
visParams = {'bands': ['B4', 'B3', 'B2'],
             'min': 0,
             'max': 0.3
            }

Now we're ready to actually plot things. First, we generate a base map centered on Santa Barbara; note the number after "zoom" below, this tells GEE how closely to zoom in on the center point. Once the base map is generated, the `addLayer` method is used to add our filtered data with the appropriate visual parameters to the map:

In [30]:
Map = geemap.Map(center=[34.42, -119.69], zoom=8)

Map

Map(center=[34.42, -119.69], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [31]:
Map.addLayer(gdat_filt, visParams)

### Sort based on metadata properties

GEE contains powerful sorting functionality, that allows you to easily identify scenes satisfying different criteria. One commonly-used filtering method is identifying the "least cloudy" scene: because clouds often interfere with many of the scientific questions people are interested in, it can be desirable to figure out which scenes are least likely to be contaminated by large cloud cover.

Recall from your perusal of the Landsat 8 metadata that one of the available "properties" is a value called CLOUD_COVER. This is a number that represents the percentage of the scene covered by clouds; applying the `sort` method based on the CLOUD_COVER property then allows us to determine which scene has the smallest fraction of cloud cover. 

The below is a compound command, which takes the filtered Santa Barbara dataset (`gdat_filt`), sorts it based on CLOUD_COVER, then selects the scene falling first in that new ordered list using the `first()` method. 

When you execute this code, it should overwrite the map displayed in your Map widget above!

In [10]:
# Use filter to extract all "non-cloudy" images: ones with less than 20% cloud cover
dat_nocld=gdat_filt.filter('CLOUD_COVER < 0.2')

# Command to extract all data over appropriate time, perform temporal averaging
dat_drght=dat_nocld.filter(ee.Filter.date('2017-12-10', '2018-12-31')).mean();

Map.addLayer(dat_drght, visParams)